In [20]:
import json
import requests
import urllib.parse
import pandas as pd

base_url = 'https://m00m3rfw1a-dsn.algolia.net/1/indexes/*/queries'
params = {
    'x-algolia-agent': 'Algolia for JavaScript (4.5.1); Browser (lite)',
    'x-algolia-api-key': '97780d6ca6d719cafd5ef2556417e2ce',
    'x-algolia-application-id': 'M00M3RFW1A'
}

request_url = f"{base_url}?{urllib.parse.urlencode(params)}"

headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-GB,en;q=0.9,en-US;q=0.8,bs;q=0.7,hr;q=0.6',
    'Connection': 'keep-alive',
    'Origin': 'https://shop.pimoroni.com',
    'Referer': 'https://shop.pimoroni.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

In [38]:
collections = [
    "Raspberry Pi",
    "Raspberry Pi Pico",
    "Feather by Adafruit",
    "Kits",
    "Components",
    "Tools",
    "Kids"
]

# "Breakouts, Boards & Electronics",

In [22]:
def request_response(request_url,headers,hits,facet):
    form_data = {
        'requests': [
            {
                'indexName': 'shop.pimoroni.com.variants',
                'query': '',
                'params': f'facetFilters=[["collections:{facet}"]]&hitsPerPage={hits}&page=0&attributesToRetrieve=["id","product_id","handle","retired","product_title","variant_title","description_first_paragraph","image","hidden_tags","price_ex_vat","compare_price_ex_vat","taxable","in_stock","rating","review_count","pre_order","special"]'
            }
        ]
    }
    response = requests.post(request_url, json=form_data, headers=headers)
    response_dict = json.loads(response.text)
    df_response = pd.DataFrame.from_dict(response_dict,orient='columns')
    return df_response

In [23]:
def hits_counter(df_response):
    hits_dict = df_response.loc[0,'results']
    df_hits = pd.json_normalize(hits_dict)
    hits = df_hits.loc[0,'nbHits']
    print("Number of hits against facet: ",hits)
    return hits

In [24]:
def base_table(request_url,headers,hits,facet):
    df_json = request_response(request_url,headers,hits,facet)
    result_dict_list = df_json.loc[0,'results']
    df_hits = pd.json_normalize(result_dict_list)
    list_of_products = df_hits.loc[0,'hits']
    df_base = pd.json_normalize(list_of_products)
    return df_base

In [25]:
def prune_cols(df):
    cols = ['id','product_id','product_title','variant_title','price_ex_vat','_highlightResult.product_type.value','_highlightResult.vendor.value','image']
    df_products = df.copy()
    df_products = df_products[cols]
    df_products.rename(columns={
        "_highlightResult.product_type.value":"product_type",
        "_highlightResult.vendor.value":"vendor",
        "price_ex_vat":"price"
    },inplace=True)
    df_products['price'] = df_products['price']*1.2
    return df_products

In [26]:
def facet_products(request_url,headers,facet):
    # Extracting Number of Product Hits First
    df_response = request_response(request_url,headers,1,facet)
    hits = hits_counter(df_response)

    # Get all products of a given facet
    df = base_table(request_url,headers,hits,facet)
    df_products = prune_cols(df)
    return df_products

Number of hits against facet:  512


In [31]:
df_products[:3]

,id,product_id,product_title,variant_title,price,product_type,vendor,image
0,31856486416467,3855360327763,Raspberry Pi 4 Model B,8GB RAM,74.94,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
1,29157087445075,3855360327763,Raspberry Pi 4 Model B,4GB RAM,54.96,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
2,29157087412307,3855360327763,Raspberry Pi 4 Model B,2GB RAM,44.88,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...


In [39]:
for i,facet in enumerate(collections):
    print(i,facet)
    if i == 0 :
        df_products = facet_products(request_url,headers,facet)
    if i != 0:
        df_facet = facet_products(request_url,headers,facet)
        df_products = pd.concat([df_products,df_facet])

1 Raspberry Pi Pico
Number of hits against facet:  138
2 Feather by Adafruit
Number of hits against facet:  130
3 Kits
Number of hits against facet:  94
4 Components
Number of hits against facet:  628
5 Tools
Number of hits against facet:  106
6 Kids
Number of hits against facet:  66


In [43]:
df_products.reset_index().drop(columns="index")

,id,product_id,product_title,variant_title,price,product_type,vendor,image
0,31856486416467,3855360327763,Raspberry Pi 4 Model B,8GB RAM,74.94,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
1,29157087445075,3855360327763,Raspberry Pi 4 Model B,4GB RAM,54.96,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
2,29157087412307,3855360327763,Raspberry Pi 4 Model B,2GB RAM,44.88,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
3,39850460086355,4488927707219,Raspberry Pi 4 + Essentials Kit,4GB RAM / UK Power Supply,83.70,Raspberry Pi,Pimoroni,https://cdn.shopify.com/s/files/1/0174/1800/pr...
4,39458414264403,10196084490,Raspberry Pi Zero W,Zero W,15.00,Raspberry Pi,Raspberry Pi,https://cdn.shopify.com/s/files/1/0174/1800/pr...
...,...,...,...,...,...,...,...,...
2031,27843989536851,3516321103955,noise:bit micro:bit Kit,None,43.80,Kit,Pimoroni,https://cdn.shopify.com/s/files/1/0174/1800/pr...
2032,21357605453907,2420157448275,enviro:bit micro:bit Kit,None,46.20,Kit,Pimoroni,https://cdn.shopify.com/s/files/1/0174/1800/pr...
2033,21357063864403,2419716620371,scroll:bit micro:bit Kit,None,46.20,Kit,Pimoroni,https://cdn.shopify.com/s/files/1/0174/1800/pr...
2034,14244431879,4150671687,Circuit Stickers LED Starter Kit,None,33.00,Kit,Chibitronics,https://cdn.shopify.com/s/files/1/0174/1800/pr...
